In [ ]:
# data manipulation
import numpy as np
import pandas as pd
import json
import ast
pd.set_option('display.max_columns', None)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Attributes

In [ ]:
Bus_fnl = pd.read_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/less_close_label/2018_yelp_business_with_2020_label(cleaned).csv')
Bus_fnl_1 = Bus_fnl.loc[~Bus_fnl['attributes'].isnull()].copy()
Bus_fnl_1['attributes_a']= Bus_fnl_1['attributes'].apply(lambda x: ast.literal_eval(x))
Bus_fnl_1 = Bus_fnl_1.drop(columns='is_open2020')
Bus_fnl_1

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,hours,attributes_a
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,30 Eglinton Avenue W,Mississauga,ON,L5R 3E7,43.605499,-79.652289,2.5,128,"{'RestaurantsReservations': 'True', 'GoodForMe...","Specialty Food, Restaurants, Dim Sum, Imported...","{'Monday': '9:0-0:0', 'Tuesday': '9:0-0:0', 'W...","{'RestaurantsReservations': 'True', 'GoodForMe..."
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"10110 Johnston Rd, Ste 15",Charlotte,NC,28210,35.092564,-80.859132,4.0,170,"{'GoodForKids': 'True', 'NoiseLevel': ""u'avera...","Sushi Bars, Restaurants, Japanese","{'Monday': '17:30-21:30', 'Wednesday': '17:30-...","{'GoodForKids': 'True', 'NoiseLevel': 'u'avera..."
2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...","{'RestaurantsTakeOut': 'True', 'BusinessParkin..."
3,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,5981 Andrews Rd,Mentor-on-the-Lake,OH,44060,41.708520,-81.359556,4.0,16,"{'RestaurantsPriceRange2': '2', 'BusinessAccep...","Italian, Restaurants, Pizza, Chicken Wings","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...","{'RestaurantsPriceRange2': '2', 'BusinessAccep..."
4,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Center Core - Food Court, Fl 3, Pittsburgh Int...",Pittsburgh,PA,15231,40.496177,-80.246011,4.0,35,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Sandwiches, Salad, Restaurants, Burgers, Comfo...",NaN,"{'RestaurantsTakeOut': 'True', 'BusinessParkin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,Church's Chicken,4800 E Flamingo Rd,Las Vegas,NV,89121,36.115214,-115.069188,2.5,24,"{'RestaurantsDelivery': 'False', 'NoiseLevel':...","Fast Food, Chicken Shop, Breakfast & Brunch, C...","{'Monday': '10:30-22:0', 'Tuesday': '10:30-22:...","{'RestaurantsDelivery': 'False', 'NoiseLevel':..."
29674,5xX1Eb4fBoUr53OJkgRHRQ,Sofia's Pizza,"5645 S Eastern Ave, Ste 1",Las Vegas,NV,89136,36.087212,-115.119135,2.0,121,"{'OutdoorSeating': 'False', 'BusinessAcceptsCr...","Restaurants, Fast Food, Pizza, Delis","{'Monday': '10:0-0:0', 'Tuesday': '10:0-0:0', ...","{'OutdoorSeating': 'False', 'BusinessAcceptsCr..."
29675,RckeGBEPQpGv00ivf6tcgw,Cozy Corner Cafe,"5901 E McKellips Rd, Ste 102",Mesa,AZ,85215,33.450360,-111.704131,4.5,195,"{'Alcohol': ""u'none'"", 'RestaurantsTakeOut': '...","Cafes, Breakfast & Brunch, Diners, Restaurants","{'Monday': '6:0-14:0', 'Tuesday': '6:0-14:0', ...","{'Alcohol': 'u'none'', 'RestaurantsTakeOut': '..."
29676,AE80fzfVnsyeHiiVhwSGMw,Deluxburger Express,3588-3964 E Sky Harbor Blvd,Phoenix,AZ,85034,33.437361,-112.001836,2.5,93,"{'WiFi': ""u'free'"", 'Alcohol': ""'none'"", 'Busi...","Restaurants, Burgers",NaN,"{'WiFi': 'u'free'', 'Alcohol': ''none'', 'Busi..."


## 1. First Level

In [ ]:
def buss_atr(atr_sr):
   from collections import defaultdict
   d1 = defaultdict(int)
   for i in atr_sr:
       for j in i.keys():
           d1[j] +=1
   df = pd.DataFrame(d1.items(),columns = ['Attribute','Count'])
   df['Pct_bus_attr'] = (df['Count']/len(atr_sr))*100
   df.sort_values(['Pct_bus_attr'],ascending = False,inplace = True)
   return df

In [ ]:
out = buss_atr(Bus_fnl_1['attributes_a'])
print('Using the Table below to find possible useful Attribute Predictors')
print(out[0:15])

Using the Table below to find possible useful Attribute Predictors
                   Attribute  Count  Pct_bus_attr
6         RestaurantsTakeOut  26616     93.396028
7     RestaurantsPriceRange2  25876     90.799354
0    RestaurantsReservations  25724     90.265984
16  RestaurantsGoodForGroups  25585     89.778230
17       RestaurantsDelivery  25534     89.599270
13               GoodForKids  25533     89.595761
8             OutdoorSeating  23969     84.107657
15         RestaurantsAttire  23943     84.016422
2            BusinessParking  23734     83.283037
10                  Ambience  23352     81.942592
11                     HasTV  23226     81.500456
9                BikeParking  22138     77.682644
14                   Alcohol  22125     77.637027
12                      WiFi  21689     76.107095
4                 NoiseLevel  20728     72.734929


In [ ]:
# Extracting the top attributes as df columns from the attribute dictionary
def extr_var(d1,i):
   if i in d1:
     return d1[i]
   else:
     return None
  
for i in list(out['Attribute'][0:15]):
   Bus_fnl_1[i] = Bus_fnl_1['attributes_a'].apply(lambda x: extr_var(x, i))

In [ ]:
# Removing redundant columns pre modeling
cols = ['name', 'address', 'city', 'state', 'postal_code',
      'latitude', 'longitude', 'stars', 'review_count', 'is_open',
      'attributes', 'categories', 'hours', 'attributes_a']
Bus_fnl_1 = Bus_fnl_1.iloc[:,list(~pd.Series(Bus_fnl_1.columns).isin(cols))]
# print 'Processed Business Data Set:'
print('')
print('Final processed Business Json with attribute predictors')
Bus_fnl_1


Final processed Business Json with attribute predictors


,business_id,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,BusinessParking,Ambience,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel
0,QXAEGFB4oINsVuTFxEYKFQ,True,2,True,True,False,True,False,u'casual',"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",False,False,u'full_bar',u'no',u'loud'
1,gnKjwL_1w79qoiV3IC_xQQ,True,2,True,True,False,True,False,'casual',"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",True,True,u'beer_and_wine',u'no',u'average'
2,1Dfx3zM-rW4n-31KeC8sJg,True,1,False,True,False,True,False,u'casual',"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'touris...",False,None,u'none',u'no',None
3,fweCYi8FmbJXHCqLnwuk8w,True,2,False,True,True,True,False,u'casual',None,None,None,True,u'none',None,None
4,1RHY4K3BD22FK7Cfftn8Mg,True,1,False,True,False,True,False,None,"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...",False,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,True,1,False,True,False,True,False,'casual',None,"{'romantic': False, 'intimate': False, 'classy...",False,True,'none','no',u'average'
29674,5xX1Eb4fBoUr53OJkgRHRQ,True,1,False,False,True,False,False,u'casual',"{'garage': False, 'street': False, 'validated'...","{'touristy': False, 'hipster': False, 'romanti...",True,True,u'none',u'no',u'quiet'
29675,RckeGBEPQpGv00ivf6tcgw,True,1,False,True,False,True,False,u'casual',"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",False,True,u'none',u'no','average'
29676,AE80fzfVnsyeHiiVhwSGMw,True,2,False,False,False,True,False,'casual',"{'garage': False, 'street': False, 'validated'...","{'romantic': False, 'intimate': False, 'classy...",False,False,'none',u'free','average'


In [ ]:
att_1st = Bus_fnl_1.drop(columns=['business_id','BusinessParking','Ambience'], inplace=False).copy()
att_1st

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel
0,True,2,True,True,False,True,False,u'casual',False,False,u'full_bar',u'no',u'loud'
1,True,2,True,True,False,True,False,'casual',True,True,u'beer_and_wine',u'no',u'average'
2,True,1,False,True,False,True,False,u'casual',False,None,u'none',u'no',None
3,True,2,False,True,True,True,False,u'casual',None,True,u'none',None,None
4,True,1,False,True,False,True,False,None,False,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,True,1,False,True,False,True,False,'casual',False,True,'none','no',u'average'
29674,True,1,False,False,True,False,False,u'casual',True,True,u'none',u'no',u'quiet'
29675,True,1,False,True,False,True,False,u'casual',False,True,u'none',u'no','average'
29676,True,2,False,False,False,True,False,'casual',False,False,'none',u'free','average'


In [ ]:
# show value_counts result for each 1st-level feature
att_value_counts = pd.DataFrame(columns=[], index=[0,1,2,3,4,5,6,7,8])
for index,item in att_1st.iteritems():  
    pd1 = pd.DataFrame({"1":item.value_counts().index, 
                        "2":item.value_counts().values})
    att_value_counts[item.name]=pd1["1"].map(str) +"："+ pd1["2"].map(str)
att_value_counts

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel
0,True：25018,2：13557,False：16555,True：21910,False：18998,True：21525,False：14814,u'casual'：13173,True：11810,True：16761,u'none'：7840,u'no'：6636,u'average'：11015
1,False：1577,1：10870,True：9149,False：3660,True：6519,False：3983,True：9129,'casual'：10080,False：11391,False：5338,u'full_bar'：6476,u'free'：6391,u'quiet'：3345
2,None：21,3：1199,None：20,None：15,None：17,None：25,None：26,u'dressy'：396,None：25,None：39,'none'：3162,'no'：4477,'average'：3172
3,NaN,4：233,NaN,NaN,NaN,NaN,NaN,'dressy'：215,NaN,NaN,u'beer_and_wine'：2173,'free'：3955,u'loud'：1474
4,NaN,None：17,NaN,NaN,NaN,NaN,NaN,'formal'：31,NaN,NaN,'full_bar'：1472,u'paid'：97,'quiet'：805
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,u'formal'：28,NaN,NaN,'beer_and_wine'：973,'paid'：79,u'very_loud'：518
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None：20,NaN,NaN,None：29,None：54,'loud'：296
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,'very_loud'：70
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None：33


Now complete some data wrangling and encoding job for the first-level attributes.

In [ ]:
# for most of the values, we replace 'True' with 1, and 'False' or None with 0
att_1st = att_1st.replace(dict.fromkeys(['True'], 1), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['False','None','none','u\'None\'','u\'none\'','no','u\'no\'', None], 0), regex=True)
att_1st

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel
0,1,2,1,1,0,1,0,u'casual',0,0,u'full_bar',0,u'loud'
1,1,2,1,1,0,1,0,'casual',1,1,u'beer_and_wine',0,u'average'
2,1,1,0,1,0,1,0,u'casual',0,0,0,0,0
3,1,2,0,1,1,1,0,u'casual',0,1,0,0,0
4,1,1,0,1,0,1,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,1,1,0,1,0,1,0,'casual',0,1,0,0,u'average'
29674,1,1,0,0,1,0,0,u'casual',1,1,0,0,u'quiet'
29675,1,1,0,1,0,1,0,u'casual',0,1,0,0,'average'
29676,1,2,0,0,0,1,0,'casual',0,0,0,u'free','average'


In [ ]:
# for 'RestaurantsAttire' column, we replace 'casual' with 1, 'dressy' with 2, and 'formal' with 3
att_1st = att_1st.replace(dict.fromkeys(['\'casual\'','u\'casual\''], 1), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'dressy\'','u\'dressy\''], 2), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'formal\'','u\'formal\''], 3), regex=True)

In [ ]:
# for 'Alcohol' column, we replace 'beer_and_wine' with 1, 'full_bar' with 2
att_1st = att_1st.replace(dict.fromkeys(['\'beer_and_wine\'','u\'beer_and_wine\''], 1), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'full_bar\'','u\'full_bar\''], 2), regex=True)

In [ ]:
# for 'NoiseLevel' column, we replace 'quiet' with 1, 'average' with 2, 'loud' with 3, and 'very_loud' with 4
att_1st = att_1st.replace(dict.fromkeys(['\'quiet\'','u\'quiet\''], 1), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'average\'','u\'average\''], 2), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'loud\'','u\'loud\''], 3), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'very_loud\'','u\'very_loud\''], 4), regex=True)

In [ ]:
# for 'WiFi' column, we replace 'free' with 1, 'paid' with 2
att_1st = att_1st.replace(dict.fromkeys(['\'free\'','u\'free\''], 1), regex=True)
att_1st = att_1st.replace(dict.fromkeys(['\'paid\'','u\'paid\''], 2), regex=True)

In [ ]:
# show value_counts result for each 1st-level feature after encoding
att_value_counts = pd.DataFrame(columns=[], index=[0,1,2,3,4])
for index,item in att_1st.iteritems():  
    pd1 = pd.DataFrame({"1":item.value_counts().index, 
                        "2":item.value_counts().values})
    att_value_counts[item.name]=pd1["1"].map(str) +"："+ pd1["2"].map(str)
att_value_counts

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel
0,1：25018,2：13557,0：19349,1：21910,0：21979,1：21525,0：19369,1：23253,0：16688,1：16761,0：17404,0：17976,2：14187
1,0：3480,1：10870,1：9149,0：6588,1：6519,0：6973,1：9129,0：4575,1：11810,0：11737,2：7948,1：10346,0：7803
2,NaN,0：2639,NaN,NaN,NaN,NaN,NaN,2：611,NaN,NaN,1：3146,2：176,1：4150
3,NaN,3：1199,NaN,NaN,NaN,NaN,NaN,3：59,NaN,NaN,NaN,NaN,3：1770
4,NaN,4：233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4：588


In [ ]:
att_1st = att_1st.join(Bus_fnl_1['business_id'])
att_1st

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel,business_id
0,1,2,1,1,0,1,0,1,0,0,2,0,3,QXAEGFB4oINsVuTFxEYKFQ
1,1,2,1,1,0,1,0,1,1,1,1,0,2,gnKjwL_1w79qoiV3IC_xQQ
2,1,1,0,1,0,1,0,1,0,0,0,0,0,1Dfx3zM-rW4n-31KeC8sJg
3,1,2,0,1,1,1,0,1,0,1,0,0,0,fweCYi8FmbJXHCqLnwuk8w
4,1,1,0,1,0,1,0,0,0,0,0,0,0,1RHY4K3BD22FK7Cfftn8Mg
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,1,1,0,1,0,1,0,1,0,1,0,0,2,Wyc49dq1r5uMqBl9bHVPAw
29674,1,1,0,0,1,0,0,1,1,1,0,0,1,5xX1Eb4fBoUr53OJkgRHRQ
29675,1,1,0,1,0,1,0,1,0,1,0,0,2,RckeGBEPQpGv00ivf6tcgw
29676,1,2,0,0,0,1,0,1,0,0,0,1,2,AE80fzfVnsyeHiiVhwSGMw


## 2. Second Level

### 2.1 Business Parking

In [ ]:
# drop none value
Bus_fnl_1['BusinessParking'] = Bus_fnl_1['BusinessParking'].replace('{}',None)
Bus_fnl_1['BusinessParking'] = Bus_fnl_1['BusinessParking'].replace('None',None)
Bus_fnl_2 = Bus_fnl_1.loc[~Bus_fnl_1['BusinessParking'].isnull()].copy()
Bus_fnl_2['BusinessParking'].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(23663,)

In [ ]:
# show the second-level attributes and frequency
Bus_fnl_2['BusinessParking']= Bus_fnl_2['BusinessParking'].apply(lambda x: ast.literal_eval(x))
out_pkg = buss_atr(Bus_fnl_2['BusinessParking'])
print('Using the Table below to find possible useful Attribute Predictors')
print(out_pkg[0:10])

Using the Table below to find possible useful Attribute Predictors
   Attribute  Count  Pct_bus_attr
0     garage  23663    100.000000
1     street  23661     99.991548
2  validated  23661     99.991548
3        lot  23661     99.991548
4      valet  23661     99.991548


In [ ]:
for i in list(out_pkg['Attribute'][0:5]):
   Bus_fnl_2[i] = Bus_fnl_2['BusinessParking'].apply(lambda x: extr_var(x, i))

In [ ]:
# Removing redundant columns pre modeling
cols = ['BusinessParking', 'business_id',	'RestaurantsTakeOut',	'RestaurantsPriceRange2',	'RestaurantsGoodForGroups',	'GoodForKids', 
        'RestaurantsReservations', 'RestaurantsDelivery',	'RestaurantsAttire',	'OutdoorSeating',	'Ambience',	'HasTV',	'Alcohol',	'NoiseLevel',	'WiFi',	'BikeParking']
att_2nd_buspk = Bus_fnl_2.iloc[:,list(~pd.Series(Bus_fnl_2.columns).isin(cols))]
# print 'Processed Business Data Set:'
print('')
print('Final processed Business Json with attribute predictors')
att_2nd_buspk.head()


Final processed Business Json with attribute predictors


,garage,street,validated,lot,valet
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False


In [ ]:
att_2nd_buspk = att_2nd_buspk.rename(columns={'garage':'BusinessParking_garage','valet':'BusinessParking_valet','street':'BusinessParking_street',
                          'lot':'BusinessParking_lot','validated':'BusinessParking_validated'})
att_2nd_buspk

,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet
0,False,False,False,True,False
1,False,False,False,True,False
2,False,False,False,False,False
3,False,False,False,False,False
4,False,False,False,False,False
...,...,...,...,...,...
29673,False,True,False,False,False
29674,False,False,False,True,False
29675,False,False,False,False,False
29676,False,False,False,False,False


In [ ]:
# show value_counts result for each 2nd-level-buspk feature after encoding
att_value_counts = pd.DataFrame(columns=[], index=[0,1])
for index,item in att_2nd_buspk.iteritems():  
    pd1 = pd.DataFrame({"1":item.value_counts().index, 
                        "2":item.value_counts().values})
    att_value_counts[item.name]=pd1["1"].map(str) +"："+ pd1["2"].map(str)
att_value_counts

,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet
0,False：22490,False：18620,False：23505,False：13376,False：23051
1,True：1173,True：5041,True：156,True：10285,True：610


In [ ]:
# so for all of the values, we replace True with 1, and False with 0
att_2nd_buspk = att_2nd_buspk.replace(True, 1)
att_2nd_buspk = att_2nd_buspk.replace(False, 0)
att_2nd_buspk

,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet
0,0,0.0,0.0,1.0,0.0
1,0,0.0,0.0,1.0,0.0
2,0,0.0,0.0,0.0,0.0
3,0,0.0,0.0,0.0,0.0
4,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...
29673,0,1.0,0.0,0.0,0.0
29674,0,0.0,0.0,1.0,0.0
29675,0,0.0,0.0,0.0,0.0
29676,0,0.0,0.0,0.0,0.0


In [ ]:
att_2nd_buspk = att_2nd_buspk.join(Bus_fnl_1['business_id'])

In [ ]:
# add encoded second-level-business-parking attributes
att_2nd_buspk = att_1st.merge(att_2nd_buspk, on='business_id', how='left')
att_2nd_buspk

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel,business_id,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet
0,1,2,1,1,0,1,0,1,0,0,2,0,3,QXAEGFB4oINsVuTFxEYKFQ,0.0,0.0,0.0,1.0,0.0
1,1,2,1,1,0,1,0,1,1,1,1,0,2,gnKjwL_1w79qoiV3IC_xQQ,0.0,0.0,0.0,1.0,0.0
2,1,1,0,1,0,1,0,1,0,0,0,0,0,1Dfx3zM-rW4n-31KeC8sJg,0.0,0.0,0.0,0.0,0.0
3,1,2,0,1,1,1,0,1,0,1,0,0,0,fweCYi8FmbJXHCqLnwuk8w,0.0,0.0,0.0,0.0,0.0
4,1,1,0,1,0,1,0,0,0,0,0,0,0,1RHY4K3BD22FK7Cfftn8Mg,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28493,1,1,0,1,0,1,0,1,0,1,0,0,2,Wyc49dq1r5uMqBl9bHVPAw,0.0,1.0,0.0,0.0,0.0
28494,1,1,0,0,1,0,0,1,1,1,0,0,1,5xX1Eb4fBoUr53OJkgRHRQ,0.0,0.0,0.0,1.0,0.0
28495,1,1,0,1,0,1,0,1,0,1,0,0,2,RckeGBEPQpGv00ivf6tcgw,0.0,0.0,0.0,0.0,0.0
28496,1,2,0,0,0,1,0,1,0,0,0,1,2,AE80fzfVnsyeHiiVhwSGMw,0.0,0.0,0.0,0.0,0.0


### 2.2 Ambience

In [ ]:
# drop none value
Bus_fnl_1['Ambience'] = Bus_fnl_1['Ambience'].replace('{}',None)
Bus_fnl_1['Ambience'] = Bus_fnl_1['Ambience'].replace('None',None)
Bus_fnl_3 = Bus_fnl_1.loc[~Bus_fnl_1['Ambience'].isnull()].copy()
Bus_fnl_3['Ambience'].shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


(23298,)

In [ ]:
# show the second-level attributes and frequency
Bus_fnl_3['Ambience']= Bus_fnl_3['Ambience'].apply(lambda x: ast.literal_eval(x))
out_amb = buss_atr(Bus_fnl_3['Ambience'])
print('Using the Table below to find possible useful Attribute Predictors')
print(out_amb[0:10])

Using the Table below to find possible useful Attribute Predictors
  Attribute  Count  Pct_bus_attr
0  romantic  23295     99.987123
1  intimate  23295     99.987123
2    classy  23295     99.987123
5  touristy  23295     99.987123
6    trendy  23295     99.987123
8    casual  23295     99.987123
7   upscale  23272     99.888402
3   hipster  23181     99.497811
4     divey  19426     83.380548


In [ ]:
for i in list(out_amb['Attribute'][0:9]):
   Bus_fnl_3[i] = Bus_fnl_3['Ambience'].apply(lambda x: extr_var(x, i))

In [ ]:
# Removing redundant columns pre modeling
cols = ['BusinessParking', 'business_id',	'RestaurantsTakeOut',	'RestaurantsPriceRange2',	'RestaurantsGoodForGroups',	'GoodForKids', 
        'RestaurantsReservations', 'RestaurantsDelivery',	'RestaurantsAttire',	'OutdoorSeating',	'Ambience',	'HasTV',	'Alcohol',	'NoiseLevel',	'WiFi',	'BikeParking']
att_2nd_amb = Bus_fnl_3.iloc[:,list(~pd.Series(Bus_fnl_3.columns).isin(cols))]
# print 'Processed Business Data Set:'
print('')
print('Final processed Business Json with attribute predictors')
att_2nd_amb.head()


Final processed Business Json with attribute predictors


,romantic,intimate,classy,touristy,trendy,casual,upscale,hipster,divey
0,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False


In [ ]:
att_2nd_amb = att_2nd_amb.rename(columns={'romantic':'Ambience_romantic','intimate':'Ambience_intimate','classy':'Ambience_classy','touristy':'Ambience_touristy',
                                          'trendy':'Ambience_trendy','casual':'Ambience_casual','upscale':'Ambience_upscale','hipster':'Ambience_hipster',
                                          'divey':'Ambience_divey'})
att_2nd_amb

,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey
0,False,False,False,False,False,True,False,False,False
1,False,False,False,False,False,True,False,False,False
2,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...
29673,False,False,False,False,False,False,False,False,False
29674,False,False,False,False,False,False,False,False,False
29675,False,False,False,False,False,True,False,False,False
29676,False,False,False,False,False,True,False,False,False


In [ ]:
# show value_counts result for each 2nd-level-amb feature after encoding
att_value_counts = pd.DataFrame(columns=[], index=[0,1])
for index,item in att_2nd_amb.iteritems():  
    pd1 = pd.DataFrame({"1":item.value_counts().index, 
                        "2":item.value_counts().values})
    att_value_counts[item.name]=pd1["1"].map(str) +"："+ pd1["2"].map(str)
att_value_counts

,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey
0,False：23027,False：22995,False：22775,False：23170,False：22038,True：12044,False：23019,False：22653,False：19036
1,True：268,True：300,True：520,True：125,True：1257,False：11251,True：253,True：528,True：390


In [ ]:
# so for all of the values, we replace True with 1, and False with 0
att_2nd_amb = att_2nd_amb.replace(True, 1)
att_2nd_amb = att_2nd_amb.replace(False, 0)
att_2nd_amb

,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...
29673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29675,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
29676,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
pd.set_option('display.max_columns', None)
att_2nd_buspk.join(att_2nd_amb)

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel,business_id,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey
0,1,2,1,1,0,1,0,1,0,0,2,0,3,QXAEGFB4oINsVuTFxEYKFQ,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,1,1,0,1,0,1,1,1,1,0,2,gnKjwL_1w79qoiV3IC_xQQ,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,1,0,1,0,1,0,1,0,0,0,0,0,1Dfx3zM-rW4n-31KeC8sJg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2,0,1,1,1,0,1,0,1,0,0,0,fweCYi8FmbJXHCqLnwuk8w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,1,0,1,0,0,0,0,0,0,0,1RHY4K3BD22FK7Cfftn8Mg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28493,1,1,0,1,0,1,0,1,0,1,0,0,2,Wyc49dq1r5uMqBl9bHVPAw,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN
28494,1,1,0,0,1,0,0,1,1,1,0,0,1,5xX1Eb4fBoUr53OJkgRHRQ,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28495,1,1,0,1,0,1,0,1,0,1,0,0,2,RckeGBEPQpGv00ivf6tcgw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
28496,1,2,0,0,0,1,0,1,0,0,0,1,2,AE80fzfVnsyeHiiVhwSGMw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
# add encoded second-level-ambience attributes to csv
att_2nd_amb = att_2nd_buspk.join(att_2nd_amb)
att_2nd_amb

,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel,business_id,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey
0,1,2,1,1,0,1,0,1,0,0,2,0,3,QXAEGFB4oINsVuTFxEYKFQ,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,1,2,1,1,0,1,0,1,1,1,1,0,2,gnKjwL_1w79qoiV3IC_xQQ,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1,1,0,1,0,1,0,1,0,0,0,0,0,1Dfx3zM-rW4n-31KeC8sJg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,2,0,1,1,1,0,1,0,1,0,0,0,fweCYi8FmbJXHCqLnwuk8w,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,1,0,1,0,1,0,0,0,0,0,0,0,1RHY4K3BD22FK7Cfftn8Mg,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28493,1,1,0,1,0,1,0,1,0,1,0,0,2,Wyc49dq1r5uMqBl9bHVPAw,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN
28494,1,1,0,0,1,0,0,1,1,1,0,0,1,5xX1Eb4fBoUr53OJkgRHRQ,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28495,1,1,0,1,0,1,0,1,0,1,0,0,2,RckeGBEPQpGv00ivf6tcgw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
28496,1,2,0,0,0,1,0,1,0,0,0,1,2,AE80fzfVnsyeHiiVhwSGMw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


# Other Non-Text Features

## Identify chain restaurants


*   Count restaurants names, select restaurants which name_counts>3 as chain restaurant
*   Label chain restaurants as 1, and others as 0.



In [ ]:
df_bus_2018 = pd.read_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/less_close_label/2018_yelp_business_with_2020_label(cleaned).csv')
df_bus = df_bus_2018[['business_id','name','categories','stars','review_count']].copy()
df_bus

,business_id,name,categories,stars,review_count
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,"Specialty Food, Restaurants, Dim Sum, Imported...",2.5,128
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"Sushi Bars, Restaurants, Japanese",4.0,170
2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,"Restaurants, Breakfast & Brunch, Mexican, Taco...",3.0,18
3,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,"Italian, Restaurants, Pizza, Chicken Wings",4.0,16
4,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",4.0,35
...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,Church's Chicken,"Fast Food, Chicken Shop, Breakfast & Brunch, C...",2.5,24
29674,5xX1Eb4fBoUr53OJkgRHRQ,Sofia's Pizza,"Restaurants, Fast Food, Pizza, Delis",2.0,121
29675,RckeGBEPQpGv00ivf6tcgw,Cozy Corner Cafe,"Cafes, Breakfast & Brunch, Diners, Restaurants",4.5,195
29676,AE80fzfVnsyeHiiVhwSGMw,Deluxburger Express,"Restaurants, Burgers",2.5,93


In [ ]:
name_counts = df_bus['name'].value_counts()
df_bus['is_chain']=df_bus['name'].isin(name_counts[name_counts>2].index)
df_bus['is_chain']=df_bus['is_chain'].astype(int)

In [ ]:
df_bus

,business_id,name,categories,stars,review_count,is_chain
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,"Specialty Food, Restaurants, Dim Sum, Imported...",2.5,128,0
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"Sushi Bars, Restaurants, Japanese",4.0,170,0
2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,"Restaurants, Breakfast & Brunch, Mexican, Taco...",3.0,18,1
3,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,"Italian, Restaurants, Pizza, Chicken Wings",4.0,16,1
4,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",4.0,35,0
...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,Church's Chicken,"Fast Food, Chicken Shop, Breakfast & Brunch, C...",2.5,24,1
29674,5xX1Eb4fBoUr53OJkgRHRQ,Sofia's Pizza,"Restaurants, Fast Food, Pizza, Delis",2.0,121,0
29675,RckeGBEPQpGv00ivf6tcgw,Cozy Corner Cafe,"Cafes, Breakfast & Brunch, Diners, Restaurants",4.5,195,0
29676,AE80fzfVnsyeHiiVhwSGMw,Deluxburger Express,"Restaurants, Burgers",2.5,93,0


## Catergories

*   Find the most 10 restaurant categories



In [ ]:
cat_counts = df_bus['categories'].value_counts()

In [ ]:
df_bus['categories'].nunique()

17228

In [ ]:
#df_bus['cat_clean'] = df_bus['categories'].str.replace(r'^Restaurants.$', '', regex=True)

In [ ]:
cat_counts[cat_counts>100]

Pizza, Restaurants                     544
Restaurants, Pizza                     522
Chinese, Restaurants                   412
Mexican, Restaurants                   389
Restaurants, Mexican                   387
Restaurants, Chinese                   384
Italian, Restaurants                   166
Restaurants, Italian                   160
Restaurants, Vietnamese                127
Vietnamese, Restaurants                125
Indian, Restaurants                    111
Thai, Restaurants                      110
Sandwiches, Restaurants, Fast Food     109
Restaurants, Thai                      109
American (Traditional), Restaurants    105
Breakfast & Brunch, Restaurants        104
Restaurants                            102
Name: categories, dtype: int64

In [ ]:
lst_cat = ['Pizza','Mexican','Chinese','Italian','American (Traditional)','Thai','Vietnamese','Indian','Breakfast & Brunch']

In [ ]:
for i in lst_cat:
    flag_cat = df_bus['categories'].str.contains(i)
    df_bus['cat_'+i]=flag_cat.astype(int)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  


In [ ]:
## Burgers, Sandwiches and Fast Food are combine as FastFood
flag_cat = df_bus['categories'].str.contains('Sandwiches|Burgers|Fast Food', regex=True)
df_bus['cat_FastFood']=flag_cat.astype(int)

### Do other types need to be set as a feature?

In [ ]:
df_bus

,business_id,name,categories,stars,review_count,is_chain,cat_Pizza,cat_Mexican,cat_Chinese,cat_Italian,cat_American (Traditional),cat_Thai,cat_Vietnamese,cat_Indian,cat_Breakfast & Brunch,cat_FastFood
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,"Specialty Food, Restaurants, Dim Sum, Imported...",2.5,128,0,0,0,1,0,0,0,0,0,0,0
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"Sushi Bars, Restaurants, Japanese",4.0,170,0,0,0,0,0,0,0,0,0,0,0
2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,"Restaurants, Breakfast & Brunch, Mexican, Taco...",3.0,18,1,0,1,0,0,0,0,0,0,1,1
3,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,"Italian, Restaurants, Pizza, Chicken Wings",4.0,16,1,1,0,0,1,0,0,0,0,0,0
4,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",4.0,35,0,0,0,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,Church's Chicken,"Fast Food, Chicken Shop, Breakfast & Brunch, C...",2.5,24,1,0,0,0,0,0,0,0,0,1,1
29674,5xX1Eb4fBoUr53OJkgRHRQ,Sofia's Pizza,"Restaurants, Fast Food, Pizza, Delis",2.0,121,0,1,0,0,0,0,0,0,0,0,1
29675,RckeGBEPQpGv00ivf6tcgw,Cozy Corner Cafe,"Cafes, Breakfast & Brunch, Diners, Restaurants",4.5,195,0,0,0,0,0,0,0,0,0,1,0
29676,AE80fzfVnsyeHiiVhwSGMw,Deluxburger Express,"Restaurants, Burgers",2.5,93,0,0,0,0,0,0,0,0,0,0,1


## Return Guest Counts


*   For each business_id, count the user_ids that appear more than once




In [ ]:
df_rev_2018=pd.read_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/Raw Matched Data 2018-2020/2018_yelp_review_matched.csv')
df_rev_2018.head()

,business_id,review_id,user_id,stars,useful,funny,cool,text,date
0,QXAEGFB4oINsVuTFxEYKFQ,6W0MQHmasK0IsaoDo4bmkw,2K62MJ4CJ19L8Tp5pRfjfQ,3.0,3,2,0,My girlfriend and I went for dinner at Emerald...,2017-01-27 21:54:30
1,QXAEGFB4oINsVuTFxEYKFQ,BeeBfUxvzD4qNX4HxrgA5g,A0kENtCCoVT3m7T35zb2Vg,3.0,0,0,0,We've always been there on a Sunday so we were...,2013-06-24 23:11:30
2,QXAEGFB4oINsVuTFxEYKFQ,A1D2kUnZ0HTroFreAheNSg,SuOLY03LW5ZcnynKhbTydA,3.0,0,0,0,"***No automatic doors, not baby friendly!*** I...",2016-01-04 12:59:22
3,QXAEGFB4oINsVuTFxEYKFQ,2pf45Stf-pNew-xgTababQ,lymyUak6KNcNKoDbK87MiQ,1.0,1,0,0,"Horrible service,\nI went there tonight with m...",2014-05-09 02:38:43
4,QXAEGFB4oINsVuTFxEYKFQ,RHhlmL07evgAdPaXQV8Omg,6vU0I5XgCv9OQHZ76rV6qw,4.0,2,1,2,One of the gauges of a good Chinese restaurant...,2011-03-21 14:39:55


In [ ]:
df_rev_2018['text'].iloc[1]

"We've always been there on a Sunday so we were hoping that Saturday dim sum would be less busy. No such luck. We were surprised that some of the dishes were cold because it was so packed; I could understand if it was empty and the carts weren't circulating but every table was full. It took a while to get drinks and other items (napkins). The dishes were not of the same quality as they had been on other visits, but they were acceptable."

In [ ]:
df_rev = df_rev_2018[['business_id','user_id']].copy()
df_rev.head()

,business_id,user_id
0,QXAEGFB4oINsVuTFxEYKFQ,2K62MJ4CJ19L8Tp5pRfjfQ
1,QXAEGFB4oINsVuTFxEYKFQ,A0kENtCCoVT3m7T35zb2Vg
2,QXAEGFB4oINsVuTFxEYKFQ,SuOLY03LW5ZcnynKhbTydA
3,QXAEGFB4oINsVuTFxEYKFQ,lymyUak6KNcNKoDbK87MiQ
4,QXAEGFB4oINsVuTFxEYKFQ,6vU0I5XgCv9OQHZ76rV6qw


In [ ]:
user_count = df_rev.groupby(['business_id','user_id'])['user_id'].count()
df_2 = user_count[user_count>1]
df_return_user = df_2.groupby('business_id').count().reset_index()
df_return_user.columns=['business_id','user_count']

In [ ]:
df_bus = df_bus.merge(df_return_user, how='left', on='business_id')
df_bus['user_count']=df_bus['user_count'].fillna(0)
df_bus

,business_id,name,categories,stars,review_count,is_chain,cat_Pizza,cat_Mexican,cat_Chinese,cat_Italian,cat_American (Traditional),cat_Thai,cat_Vietnamese,cat_Indian,cat_Breakfast & Brunch,cat_FastFood,user_count
0,QXAEGFB4oINsVuTFxEYKFQ,Emerald Chinese Restaurant,"Specialty Food, Restaurants, Dim Sum, Imported...",2.5,128,0,0,0,1,0,0,0,0,0,0,0,5.0
1,gnKjwL_1w79qoiV3IC_xQQ,Musashi Japanese Restaurant,"Sushi Bars, Restaurants, Japanese",4.0,170,0,0,0,0,0,0,0,0,0,0,0,7.0
2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,"Restaurants, Breakfast & Brunch, Mexican, Taco...",3.0,18,1,0,1,0,0,0,0,0,0,1,1,1.0
3,fweCYi8FmbJXHCqLnwuk8w,Marco's Pizza,"Italian, Restaurants, Pizza, Chicken Wings",4.0,16,1,1,0,0,1,0,0,0,0,0,0,2.0
4,1RHY4K3BD22FK7Cfftn8Mg,Marathon Diner,"Sandwiches, Salad, Restaurants, Burgers, Comfo...",4.0,35,0,0,0,0,0,0,0,0,0,0,1,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,Church's Chicken,"Fast Food, Chicken Shop, Breakfast & Brunch, C...",2.5,24,1,0,0,0,0,0,0,0,0,1,1,0.0
29674,5xX1Eb4fBoUr53OJkgRHRQ,Sofia's Pizza,"Restaurants, Fast Food, Pizza, Delis",2.0,121,0,1,0,0,0,0,0,0,0,0,1,3.0
29675,RckeGBEPQpGv00ivf6tcgw,Cozy Corner Cafe,"Cafes, Breakfast & Brunch, Diners, Restaurants",4.5,195,0,0,0,0,0,0,0,0,0,1,0,5.0
29676,AE80fzfVnsyeHiiVhwSGMw,Deluxburger Express,"Restaurants, Burgers",2.5,93,0,0,0,0,0,0,0,0,0,0,1,0.0


## Check in Counts

In [ ]:
df_checkin_2018=pd.read_json('/content/drive/MyDrive/SEAS 6402 Project/Data/JSON/2018_yelp_checkin.json',lines=True) 
df_checkin_2018.head()

,business_id,date
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016..."
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012..."
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015..."
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010..."


In [ ]:
df_checkin_2018.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161950 entries, 0 to 161949
Data columns (total 2 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   business_id  161950 non-null  object
 1   date         161950 non-null  object
dtypes: object(2)
memory usage: 2.5+ MB


In [ ]:
df_checkin_2018['checkin_count']=df_checkin_2018['date'].str.split(',').apply(len)

In [ ]:
df_checkin_2018.head()

,business_id,date,checkin_count
0,--1UhMGODdWsrMastO9DZw,"2016-04-26 19:49:16, 2016-08-30 18:36:57, 2016...",6
1,--6MefnULPED_I942VcFNA,"2011-06-04 18:22:23, 2011-07-23 23:51:33, 2012...",167
2,--7zmmkVg-IMGaXbuVd0SQ,"2014-12-29 19:25:50, 2015-01-17 01:49:14, 2015...",180
3,--8LPVSo5i0Oo61X01sV9A,2016-07-08 16:43:30,1
4,--9QQLMTbFzLJ_oT-ON3Xw,"2010-06-26 17:39:07, 2010-08-01 20:06:21, 2010...",37


In [ ]:
df_bus = df_bus.merge(df_checkin_2018, how='left', on='business_id')

In [ ]:
df_bus = df_bus.drop(['date','categories','name'],axis=1).copy()

In [ ]:
df_bus

,business_id,stars,review_count,is_chain,cat_Pizza,cat_Mexican,cat_Chinese,cat_Italian,cat_American (Traditional),cat_Thai,cat_Vietnamese,cat_Indian,cat_Breakfast & Brunch,cat_FastFood,user_count,checkin_count
0,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,0,0,0,1,0,0,0,0,0,0,0,5.0,455.0
1,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,0,0,0,0,0,0,0,0,0,0,0,7.0,721.0
2,1Dfx3zM-rW4n-31KeC8sJg,3.0,18,1,0,1,0,0,0,0,0,0,1,1,1.0,90.0
3,fweCYi8FmbJXHCqLnwuk8w,4.0,16,1,1,0,0,1,0,0,0,0,0,0,2.0,5.0
4,1RHY4K3BD22FK7Cfftn8Mg,4.0,35,0,0,0,0,0,0,0,0,0,0,1,2.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,2.5,24,1,0,0,0,0,0,0,0,0,1,1,0.0,51.0
29674,5xX1Eb4fBoUr53OJkgRHRQ,2.0,121,0,1,0,0,0,0,0,0,0,0,1,3.0,30.0
29675,RckeGBEPQpGv00ivf6tcgw,4.5,195,0,0,0,0,0,0,0,0,0,1,0,5.0,469.0
29676,AE80fzfVnsyeHiiVhwSGMw,2.5,93,0,0,0,0,0,0,0,0,0,0,1,0.0,169.0


In [ ]:
df_bus.shape

(29678, 16)

## Location - Personal Income

* For US, we found the annual per capita personal income by county ([from 
Bureau of Economic Analysis](https://apps.bea.gov/regional/downloadzip.cfm)). Then we used zipcode from our dataset to find county correspondingly ([use dataset from kaggle](https://www.kaggle.com/datasets/danofer/zipcodes-county-fips-crosswalk)). 
* For Canada, however, we couldn't find such of data based on county or city. So we applyied the data based on province ([from Statistics Canada](https://www150.statcan.gc.ca/t1/tbl1/en/tv.action?pid=1110023901)).


In [ ]:
ca = pd.read_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/Income/ca_province_per capita personal income.csv', 
                 encoding = 'unicode_escape', engine ='python')
ca = ca[['business_id','per capita personal income (dollars)']]
ca['business_id'].nunique()

12067

In [ ]:
us = pd.read_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/Income/us_county_per capita personal income.csv', 
                 encoding = 'unicode_escape', engine ='python')
us = us[['business_id','per capita personal income (dollars)']]
us['business_id'].nunique()

17611

In [ ]:
income = pd.concat([us,ca], axis=0, ignore_index=True)
income

,business_id,per capita personal income (dollars)
0,-U7tvCtaraTQ9b0zBhpBMA,35696
1,-jEyZSptICvvaNhuur9rUg,35696
2,-hzQQ8tbXThpp0k5E_y1Dw,35696
3,-R_u_4Ya8ceUN7qDlvIQTg,35696
4,-PARO4rKPyOBgra18yANYQ,47606
...,...,...
29673,__pTR2Zcmofr_1FpAaQZhA,37400
29674,__oEHyLE8wv92Q3zsfh3cQ,42400
29675,__iqJ91sPngnwEa3nIQP8Q,35700
29676,__fyRzU8kL6HkVV3wgxfmQ,35700


# Merge All of Non-text Features

In [ ]:
bus_ntf = df_bus.merge(att_2nd_amb, on='business_id', how='left')
bus_ntf

,business_id,stars,review_count,is_chain,cat_Pizza,cat_Mexican,cat_Chinese,cat_Italian,cat_American (Traditional),cat_Thai,cat_Vietnamese,cat_Indian,cat_Breakfast & Brunch,cat_FastFood,user_count,checkin_count,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey
0,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,0,0,0,1,0,0,0,0,0,0,0,5.0,455.0,1.0,2,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
1,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,0,0,0,0,0,0,0,0,0,0,0,7.0,721.0,1.0,2,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,1Dfx3zM-rW4n-31KeC8sJg,3.0,18,1,0,1,0,0,0,0,0,0,1,1,1.0,90.0,1.0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,fweCYi8FmbJXHCqLnwuk8w,4.0,16,1,1,0,0,1,0,0,0,0,0,0,2.0,5.0,1.0,2,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1RHY4K3BD22FK7Cfftn8Mg,4.0,35,0,0,0,0,0,0,0,0,0,0,1,2.0,21.0,1.0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,2.5,24,1,0,0,0,0,0,0,0,0,1,1,0.0,51.0,1.0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN
29674,5xX1Eb4fBoUr53OJkgRHRQ,2.0,121,0,1,0,0,0,0,0,0,0,0,1,3.0,30.0,1.0,1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29675,RckeGBEPQpGv00ivf6tcgw,4.5,195,0,0,0,0,0,0,0,0,0,1,0,5.0,469.0,1.0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
29676,AE80fzfVnsyeHiiVhwSGMw,2.5,93,0,0,0,0,0,0,0,0,0,0,1,0.0,169.0,1.0,2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [ ]:
bus_ntf = bus_ntf.merge(income, on='business_id', how='left')
bus_ntf

,business_id,stars,review_count,is_chain,cat_Pizza,cat_Mexican,cat_Chinese,cat_Italian,cat_American (Traditional),cat_Thai,cat_Vietnamese,cat_Indian,cat_Breakfast & Brunch,cat_FastFood,user_count,checkin_count,RestaurantsTakeOut,RestaurantsPriceRange2,RestaurantsReservations,RestaurantsGoodForGroups,RestaurantsDelivery,GoodForKids,OutdoorSeating,RestaurantsAttire,HasTV,BikeParking,Alcohol,WiFi,NoiseLevel,BusinessParking_garage,BusinessParking_street,BusinessParking_validated,BusinessParking_lot,BusinessParking_valet,Ambience_romantic,Ambience_intimate,Ambience_classy,Ambience_touristy,Ambience_trendy,Ambience_casual,Ambience_upscale,Ambience_hipster,Ambience_divey,per capita personal income (dollars)
0,QXAEGFB4oINsVuTFxEYKFQ,2.5,128,0,0,0,1,0,0,0,0,0,0,0,5.0,455.0,1.0,2,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,37400
1,gnKjwL_1w79qoiV3IC_xQQ,4.0,170,0,0,0,0,0,0,0,0,0,0,0,7.0,721.0,1.0,2,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,1.0,0.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,60804
2,1Dfx3zM-rW4n-31KeC8sJg,3.0,18,1,0,1,0,0,0,0,0,0,1,1,1.0,90.0,1.0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,47606
3,fweCYi8FmbJXHCqLnwuk8w,4.0,16,1,1,0,0,1,0,0,0,0,0,0,2.0,5.0,1.0,2,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,69949
4,1RHY4K3BD22FK7Cfftn8Mg,4.0,35,0,0,0,0,0,0,0,0,0,0,1,2.0,21.0,1.0,1,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,63371
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,2.5,24,1,0,0,0,0,0,0,0,0,1,1,0.0,51.0,1.0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,NaN,35696
29674,5xX1Eb4fBoUr53OJkgRHRQ,2.0,121,0,1,0,0,0,0,0,0,0,0,1,3.0,30.0,1.0,1,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,35696
29675,RckeGBEPQpGv00ivf6tcgw,4.5,195,0,0,0,0,0,0,0,0,0,1,0,5.0,469.0,1.0,1,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,47606
29676,AE80fzfVnsyeHiiVhwSGMw,2.5,93,0,0,0,0,0,0,0,0,0,0,1,0.0,169.0,1.0,2,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,47606


In [ ]:
target = pd.read_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/less_close_label/2018_yelp_business_with_2020_label(cleaned).csv')
target.set_index("business_id", inplace = True)
target = target[['is_open2020']]
target = target.reset_index()
target

,business_id,is_open2020
0,QXAEGFB4oINsVuTFxEYKFQ,1.0
1,gnKjwL_1w79qoiV3IC_xQQ,1.0
2,1Dfx3zM-rW4n-31KeC8sJg,1.0
3,fweCYi8FmbJXHCqLnwuk8w,1.0
4,1RHY4K3BD22FK7Cfftn8Mg,1.0
...,...,...
29673,Wyc49dq1r5uMqBl9bHVPAw,1.0
29674,5xX1Eb4fBoUr53OJkgRHRQ,1.0
29675,RckeGBEPQpGv00ivf6tcgw,1.0
29676,AE80fzfVnsyeHiiVhwSGMw,1.0


In [ ]:
bus_ntf = bus_ntf.merge(target, on='business_id', how='left')

In [ ]:
bus_ntf.isnull().sum()

business_id                                 0
stars                                       0
review_count                                0
is_chain                                    0
cat_Pizza                                   0
cat_Mexican                                 0
cat_Chinese                                 0
cat_Italian                                 0
cat_American (Traditional)                  0
cat_Thai                                    0
cat_Vietnamese                              0
cat_Indian                                  0
cat_Breakfast & Brunch                      0
cat_FastFood                                0
user_count                                  0
checkin_count                             582
RestaurantsTakeOut                       1180
RestaurantsPriceRange2                   1180
RestaurantsReservations                  1180
RestaurantsGoodForGroups                 1180
RestaurantsDelivery                      1180
GoodForKids                       

In [ ]:
# fill nan with 0
bus_ntf = bus_ntf.fillna(0)

In [ ]:
bus_ntf.isnull().sum()

business_id                             0
stars                                   0
review_count                            0
is_chain                                0
cat_Pizza                               0
cat_Mexican                             0
cat_Chinese                             0
cat_Italian                             0
cat_American (Traditional)              0
cat_Thai                                0
cat_Vietnamese                          0
cat_Indian                              0
cat_Breakfast & Brunch                  0
cat_FastFood                            0
user_count                              0
checkin_count                           0
RestaurantsTakeOut                      0
RestaurantsPriceRange2                  0
RestaurantsReservations                 0
RestaurantsGoodForGroups                0
RestaurantsDelivery                     0
GoodForKids                             0
OutdoorSeating                          0
RestaurantsAttire                 

In [ ]:
bus_ntf.to_csv('/content/drive/MyDrive/SEAS 6402 Project/Data/CSV/less_close_label/non_text_feature.csv', index=False)